In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=59cb0648e6bc43eb2e8f3e2adfca739d5eac17e9748de9400a6822e193ac64c7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.appName('DataFrame').getOrCreate()

In [8]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## NBA

In [9]:
players = spark.read.load("/content/drive/MyDrive/01 - Laboral/Accenture/Data academy/Data engineering/players_(1).csv", format='csv', sep=',', header="true")
teams = spark.read.load("/content/drive/MyDrive/01 - Laboral/Accenture/Data academy/Data engineering/teams_(1).csv", format='csv', sep=',', header="true")
games = spark.read.load("/content/drive/MyDrive/01 - Laboral/Accenture/Data academy/Data engineering/games_(1).csv", format='csv', sep=',', header="true")
games_details = spark.read.load("/content/drive/MyDrive/01 - Laboral/Accenture/Data academy/Data engineering/games_details.csv", format='csv', sep=',', header="true")
ranking = spark.read.load("/content/drive/MyDrive/01 - Laboral/Accenture/Data academy/Data engineering/ranking.csv", format='csv', sep=',', header="true")

In [38]:
players.show(5)

+----------------+----------+---------+------+
|     PLAYER_NAME|   TEAM_ID|PLAYER_ID|SEASON|
+----------------+----------+---------+------+
|   Royce O'Neale|1610612762|  1626220|  2019|
|Bojan Bogdanovic|1610612762|   202711|  2019|
|     Rudy Gobert|1610612762|   203497|  2019|
|Donovan Mitchell|1610612762|  1628378|  2019|
|     Mike Conley|1610612762|   201144|  2019|
+----------------+----------+---------+------+
only showing top 5 rows



In [39]:
teams.show(5)

+---------+----------+--------+--------+------------+---------+-----------+-----------+--------------------+-------------+---------------+--------------+-------------+------------------+
|LEAGUE_ID|   TEAM_ID|MIN_YEAR|MAX_YEAR|ABBREVIATION| NICKNAME|YEARFOUNDED|       CITY|               ARENA|ARENACAPACITY|          OWNER|GENERALMANAGER|    HEADCOACH|DLEAGUEAFFILIATION|
+---------+----------+--------+--------+------------+---------+-----------+-----------+--------------------+-------------+---------------+--------------+-------------+------------------+
|       00|1610612737|    1949|    2019|         ATL|    Hawks|       1949|    Atlanta|    State Farm Arena|        18729|   Tony Ressler|Travis Schlenk| Lloyd Pierce|     Erie Bayhawks|
|       00|1610612738|    1946|    2019|         BOS|  Celtics|       1946|     Boston|           TD Garden|        18624|  Wyc Grousbeck|   Danny Ainge| Brad Stevens|   Maine Red Claws|
|       00|1610612740|    2002|    2019|         NOP| Pelicans|  

In [40]:
games.show(5)

+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|GAME_DATE_EST| GAME_ID|GAME_STATUS_TEXT|HOME_TEAM_ID|VISITOR_TEAM_ID|SEASON|TEAM_ID_home|PTS_home|FG_PCT_home|FT_PCT_home|FG3_PCT_home|AST_home|REB_home|TEAM_ID_away|PTS_away|FG_PCT_away|FT_PCT_away|FG3_PCT_away|AST_away|REB_away|HOME_TEAM_WINS|
+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|   2022-03-12|22101005|           Final|  1610612748|     1610612750|  2021|  1610612748|     104|      0.398|       0.76|       0.333|      23|      53|  1610612750|     113|      0.422|      0.875|       0.357|      21|      46|             0|
|   2022-03-

In [10]:
players.createOrReplaceTempView('players')
teams.createOrReplaceTempView('teams')
games.createOrReplaceTempView('games')
games_details.createOrReplaceTempView('games_details')
ranking.createOrReplaceTempView('ranking')


In [11]:
games_clean = spark.sql("""
  SELECT
    *
  FROM
    games
  WHERE
    PTS_home IS NOT NULL

""")

games_clean.createOrReplaceTempView('games_clean')

In [23]:
games_clean.show(5)

+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|GAME_DATE_EST| GAME_ID|GAME_STATUS_TEXT|HOME_TEAM_ID|VISITOR_TEAM_ID|SEASON|TEAM_ID_home|PTS_home|FG_PCT_home|FT_PCT_home|FG3_PCT_home|AST_home|REB_home|TEAM_ID_away|PTS_away|FG_PCT_away|FT_PCT_away|FG3_PCT_away|AST_away|REB_away|HOME_TEAM_WINS|
+-------------+--------+----------------+------------+---------------+------+------------+--------+-----------+-----------+------------+--------+--------+------------+--------+-----------+-----------+------------+--------+--------+--------------+
|   2022-03-12|22101005|           Final|  1610612748|     1610612750|  2021|  1610612748|     104|      0.398|       0.76|       0.333|      23|      53|  1610612750|     113|      0.422|      0.875|       0.357|      21|      46|             0|
|   2022-03-

In [28]:
temporadas_unicas = spark.sql("""
SELECT
  PLAYER_NAME,
  COUNT(DISTINCT SEASON) AS TEMPORADAS_DISTINCT
FROM
  players
GROUP BY
  PLAYER_NAME
ORDER BY
  TEMPORADAS_DISTINCT DESC;

""")

temporadas_unicas.createOrReplaceTempView('temporadas_unicas')

In [32]:
temporadas_unicas.show()

+---------------+-------------------+
|    PLAYER_NAME|TEMPORADAS_DISTINCT|
+---------------+-------------------+
|    Ian Mahinmi|                 11|
|    Kyle Korver|                 11|
|     Taj Gibson|                 11|
|           Nene|                 11|
|       Rudy Gay|                 11|
|     J.J. Barea|                 11|
|Wayne Ellington|                 11|
|    Patty Mills|                 11|
|   Courtney Lee|                 11|
|     Al Horford|                 11|
|    Danny Green|                 11|
|    Eric Gordon|                 11|
|   James Harden|                 11|
|     Kevin Love|                 11|
|   Lou Williams|                 11|
|    George Hill|                 11|
|      Pau Gasol|                 11|
|   Jared Dudley|                 11|
|    Serge Ibaka|                 11|
|  D.J. Augustin|                 11|
+---------------+-------------------+
only showing top 20 rows



# 2- Calcular el promedio de puntos anotados por los equipos en casa y fuera de casa

In [53]:
home = spark.sql("""
  SELECT
    HOME_TEAM_ID AS TEAM_ID,
    ROUND(AVG(PTS_home),2) AS PTS_HOME_AVG
  FROM
    games_clean
  GROUP BY
    TEAM_ID
"""
)

home.createOrReplaceTempView('home')

visitor = spark.sql("""
  SELECT
    VISITOR_TEAM_ID AS TEAM_ID,
    ROUND(AVG(PTS_away),2) AS PTS_VISITOR_AVG
  FROM
    games_clean
  GROUP BY
    TEAM_ID
"""
)

visitor.createOrReplaceTempView('visitor')

puntos_totales = spark.sql("""
  SELECT
    t.NICKNAME,
    h.PTS_HOME_AVG,
    v.PTS_VISITOR_AVG
  FROM
    home AS h
  JOIN
    visitor AS v
    ON h.TEAM_ID = v.TEAM_ID
  JOIN
    teams AS t
    ON h.TEAM_ID = t.TEAM_ID
"""
)

puntos_totales.createOrReplaceTempView('puntos_totales')

In [54]:
puntos_totales.show()

+---------+------------+---------------+
| NICKNAME|PTS_HOME_AVG|PTS_VISITOR_AVG|
+---------+------------+---------------+
|    76ers|       101.4|          98.89|
|    Magic|      100.99|          98.57|
|    Spurs|      103.87|         100.32|
| Pelicans|      101.75|          99.18|
|    Kings|       104.9|         101.33|
|   Lakers|      104.23|         102.15|
| Clippers|       103.7|         101.54|
|    Bucks|      103.72|         100.61|
|    Hawks|       102.4|          98.79|
|    Bulls|      100.24|           98.8|
| Warriors|      109.25|         105.28|
|  Pistons|       99.46|          96.39|
|     Heat|      101.86|          98.05|
|  Celtics|      102.11|         100.35|
|Grizzlies|      100.92|          98.64|
|Mavericks|      104.71|         101.03|
|  Wizards|      103.83|          99.98|
|  Rockets|      104.47|         102.68|
|  Raptors|      103.79|         100.99|
|     Nets|       100.5|          99.26|
+---------+------------+---------------+
only showing top

# 3- Encontrar los jugadores que tienen un promedio de más de 10 puntos por partido

In [63]:
spark.sql("""
  SELECT
    PLAYER_NAME,
    ROUND(AVG(PTS),2) AS AVG_PTS
  FROM
    games_details
  GROUP BY
    PLAYER_NAME
  HAVING AVG_PTS > 10
  ORDER BY
    AVG_PTS DESC
"""
).show()

+--------------------+-------+
|         PLAYER_NAME|AVG_PTS|
+--------------------+-------+
|        Kevin Durant|  26.96|
|        LeBron James|  26.83|
|         Kobe Bryant|  26.62|
|         Luka Doncic|  26.05|
|       Allen Iverson|  25.86|
|     Zion Williamson|  25.62|
|         Joel Embiid|  24.97|
|          Trae Young|  24.68|
|        James Harden|  24.44|
|      Damian Lillard|  24.31|
|       Stephen Curry|  24.14|
|    Donovan Mitchell|  23.75|
|       Anthony Davis|  23.53|
|        Devin Booker|  23.03|
|        Kyrie Irving|  22.66|
|  Karl-Anthony Towns|  22.57|
|   Russell Westbrook|  22.54|
|     Carmelo Anthony|  22.25|
|Giannis Antetokou...|   21.7|
|        Bradley Beal|  21.68|
+--------------------+-------+
only showing top 20 rows



# 4- Ordenar los equipos por su porcentaje de victorias en la conferencia Oeste

In [12]:
spark.sql("""
SELECT
  b.NICKNAME,
  a.W_PCT
FROM
  ranking a
JOIN
  teams b
  ON a.TEAM_ID = b.TEAM_ID
WHERE
  CONFERENCE = 'West'
ORDER BY
  W_PCT DESC
""").show()

+------------+-----+
|    NICKNAME|W_PCT|
+------------+-----+
|        Jazz|  1.0|
|   Grizzlies|  1.0|
|        Jazz|  1.0|
|     Nuggets|  1.0|
|        Jazz|  1.0|
|Timberwolves|  1.0|
|    Warriors|  1.0|
|        Jazz|  1.0|
|    Warriors|  1.0|
|        Jazz|  1.0|
|    Warriors|  1.0|
|        Jazz|  1.0|
|    Warriors|  1.0|
|        Jazz|  1.0|
|     Nuggets|  1.0|
|    Warriors|  1.0|
|   Grizzlies|  1.0|
|Timberwolves|  1.0|
|        Jazz|  1.0|
|     Nuggets|  1.0|
+------------+-----+
only showing top 20 rows



# 5- Identificar el jugador que ha jugado en más temporadas consecutivas

In [83]:
unique_player_season = spark.sql ("""
  SELECT DISTINCT
    PLAYER_ID,
    SEASON
  FROM
    players
"""
  )

unique_player_season.createOrReplaceTempView('unique_player_season')

season_difference = spark.sql ("""
  SELECT
    PLAYER_ID,
    SEASON,
    SEASON - LAG(SEASON, 1) OVER (PARTITION BY PLAYER_ID ORDER BY SEASON) SEASON_DIFF
  FROM
    unique_player_season
"""
)

season_difference.createOrReplaceTempView('season_difference')

continuidad = spark.sql ("""
  SELECT
    PLAYER_ID,
    SUM(SEASON_DIFF) AS TEMPORADAS_CONTINUAS
  FROM
    season_difference
  WHERE
    SEASON_DIFF = 1
  GROUP BY
    PLAYER_ID
"""
)

continuidad.createOrReplaceTempView('continuidad')

temporadas_consecutivas = spark.sql("""
SELECT
  a.PLAYER_ID,
  b.PLAYER_NAME,
  a.TEMPORADAS_CONTINUAS AS TOP1
FROM
  continuidad a
JOIN
  (SELECT DISTINCT PLAYER_ID, PLAYER_NAME FROM players) b
  ON a.PLAYER_ID = b.PLAYER_ID
WHERE
  TEMPORADAS_CONTINUAS = (SELECT MAX(TEMPORADAS_CONTINUAS) FROM continuidad)
""")

temporadas_consecutivas.createOrReplaceTempView('temporadas_consecutivas')

temporadas_consecutivas.show()

+---------+-----------------+----+
|PLAYER_ID|      PLAYER_NAME|TOP1|
+---------+-----------------+----+
|   101107|  Marvin Williams|10.0|
|   101108|       Chris Paul|10.0|
|   101133|      Ian Mahinmi|10.0|
|   101139|         CJ Miles|10.0|
|   101141|   Ersan Ilyasova|10.0|
|   101150|     Lou Williams|10.0|
|     1713|     Vince Carter|10.0|
|   200746|LaMarcus Aldridge|10.0|
|   200752|         Rudy Gay|10.0|
|   200755|        JJ Redick|10.0|
|   200757|  Thabo Sefolosha|10.0|
|   200765|      Rajon Rondo|10.0|
|   200768|       Kyle Lowry|10.0|
|   200794|     Paul Millsap|10.0|
|   200826|       J.J. Barea|10.0|
|   201142|     Kevin Durant|10.0|
|   201143|       Al Horford|10.0|
|   201144|      Mike Conley|10.0|
|   201152|   Thaddeus Young|10.0|
|   201158|  Marco Belinelli|10.0|
+---------+-----------------+----+
only showing top 20 rows



# 6- Calcular el porcentaje de juegos ganados en casa para cada equipo

In [84]:
victorias = spark.sql("""
  SELECT
    HOME_TEAM_ID,
    COUNT(HOME_TEAM_ID) VICTORIAS
  FROM
    games_clean
  WHERE
    HOME_TEAM_WINS = 1
  GROUP BY
    HOME_TEAM_ID
  """
)

victorias.createOrReplaceTempView('victorias')

total = spark.sql("""
  SELECT
    HOME_TEAM_ID,
    COUNT(HOME_TEAM_ID) TOTAL
  FROM
    games_clean
  GROUP BY
    HOME_TEAM_ID
"""
)

total.createOrReplaceTempView('total')

ganados = spark.sql("""
  SELECT
    a.HOME_TEAM_ID,
    c.NICKNAME,
    ROUND(((VICTORIAS / TOTAL)*100), 2) AS PERC_VICTORIAS
  FROM
    victorias a
  JOIN
    total b
    ON a.HOME_TEAM_ID = b.HOME_TEAM_ID
  JOIN
    teams c
    ON a.HOME_TEAM_ID = c.TEAM_ID
  ORDER BY
    PERC_VICTORIAS DESC
"""
)

ganados.createOrReplaceTempView('ganados')

ganados.show()

+------------+-------------+--------------+
|HOME_TEAM_ID|     NICKNAME|PERC_VICTORIAS|
+------------+-------------+--------------+
|  1610612759|        Spurs|          74.4|
|  1610612743|      Nuggets|         67.85|
|  1610612744|     Warriors|         66.59|
|  1610612748|         Heat|         66.27|
|  1610612762|         Jazz|         66.11|
|  1610612742|    Mavericks|         65.37|
|  1610612745|      Rockets|          64.6|
|  1610612738|      Celtics|         63.78|
|  1610612754|       Pacers|         62.73|
|  1610612757|Trail Blazers|         61.04|
|  1610612760|      Thunder|         60.51|
|  1610612747|       Lakers|         60.19|
|  1610612746|     Clippers|         59.95|
|  1610612739|    Cavaliers|         59.91|
|  1610612761|      Raptors|         59.39|
|  1610612763|    Grizzlies|         59.25|
|  1610612741|        Bulls|         58.16|
|  1610612765|      Pistons|         58.03|
|  1610612749|        Bucks|         57.71|
|  1610612756|         Suns|    

# 7- Encontrar los jugadores que han tenido un aumento significativo en el promedio de puntos por partido


In [85]:
games_details.show(5)

+--------+----------+-----------------+---------+---------+------------------+------------+--------------+-------+-----+---+----+------+----+----+-------+---+---+------+----+----+----+---+---+---+---+---+----+----------+
| GAME_ID|   TEAM_ID|TEAM_ABBREVIATION|TEAM_CITY|PLAYER_ID|       PLAYER_NAME|    NICKNAME|START_POSITION|COMMENT|  MIN|FGM| FGA|FG_PCT|FG3M|FG3A|FG3_PCT|FTM|FTA|FT_PCT|OREB|DREB| REB|AST|STL|BLK| TO| PF| PTS|PLUS_MINUS|
+--------+----------+-----------------+---------+---------+------------------+------------+--------------+-------+-----+---+----+------+----+----+-------+---+---+------+----+----+----+---+---+---+---+---+----+----------+
|22101005|1610612750|              MIN|Minnesota|  1630162|   Anthony Edwards|     Anthony|             F|   NULL|36:22|4.0|10.0|   0.4| 3.0| 8.0|  0.375|4.0|4.0|   1.0| 0.0| 8.0| 8.0|5.0|3.0|1.0|1.0|1.0|15.0|       5.0|
|22101005|1610612750|              MIN|Minnesota|  1630183|   Jaden McDaniels|       Jaden|             F|   NULL|23

# 8- Calcular la diferencia en el porcentaje de victorias entre temporadas para cada equipo

In [13]:
# total de partidos ganados por equipo por año

victorias_year = spark.sql("""
  SELECT
    HOME_TEAM_ID AS TEAM_ID,
    SEASON,
    SUM(HOME_TEAM_WINS) AS VICTORIAS

  FROM
    games_clean
  WHERE
    HOME_TEAM_WINS = 1
  GROUP BY
    TEAM_ID,
    SEASON

UNION ALL

  SELECT
    VISITOR_TEAM_ID AS TEAM_ID,
    SEASON,
    SUM(1) AS VICTORIAS

  FROM
    games_clean
  WHERE
    HOME_TEAM_WINS = 0
  GROUP BY
    TEAM_ID,
    SEASON
"""
)

victorias_year.createOrReplaceTempView('victorias_year')

# suma las victorias como home y visitor para cada equipo para cada año

total_victorias_year = spark.sql("""
  SELECT
    TEAM_ID,
    SEASON,
    SUM(VICTORIAS)
  FROM
    victorias_year
  GROUP BY
    TEAM_ID,
    SEASON
"""
)

total_victorias_year.createOrReplaceTempView('total_victorias_year')

# total de partidos por equipo por año

total_year_home_visitor = spark.sql("""
  SELECT
    HOME_TEAM_ID AS TEAM_ID,
    SEASON,
    COUNT(HOME_TEAM_ID) AS TOTAL
  FROM
    games_clean
  GROUP BY
    TEAM_ID,
    SEASON

  UNION ALL

  SELECT
    VISITOR_TEAM_ID AS TEAM_ID,
    SEASON,
    COUNT(VISITOR_TEAM_ID) AS TOTAL
  FROM
    games_clean
  GROUP BY
    TEAM_ID,
    SEASON
""")

total_year_home_visitor.createOrReplaceTempView('total_year_home_visitor')

total_year = spark.sql("""
  SELECT
    TEAM_ID,
    SEASON,
    SUM(TOTAL) AS TOTAL
  FROM
    total_year_home_visitor
  GROUP BY
    TEAM_ID,
    SEASON
""")

total_year.createOrReplaceTempView('total_year')

#- porcentaje de victorias por equipo por año
perc_victorias_year = spark.sql("""
  SELECT
    a.TEAM_ID,
    a.SEASON,
    a.VICTORIAS / b.TOTAL AS PERC_VICTORIAS
  FROM
    victorias_year a
  LEFT JOIN
    total_year b
    ON
      a.TEAM_ID = b.TEAM_ID
      AND a.SEASON = b.SEASON
""")

perc_victorias_year.createOrReplaceTempView('perc_victorias_year')

victorias_dif = spark.sql("""
  SELECT
    a.TEAM_ID,
    b.NICKNAME,
    a.SEASON,
    ROUND((a.PERC_VICTORIAS - LAG(a.PERC_VICTORIAS, 1) OVER (PARTITION BY NICKNAME ORDER BY SEASON)),2) AS PERC_DIFF
  FROM
    perc_victorias_year a
  JOIN
    teams b
    ON a.TEAM_ID = b.TEAM_ID
""")

victorias_dif.createOrReplaceTempView('victorias_dif')

victorias_dif.show()

+----------+--------+------+---------+
|   TEAM_ID|NICKNAME|SEASON|PERC_DIFF|
+----------+--------+------+---------+
|1610612755|   76ers|  2003|     NULL|
|1610612755|   76ers|  2003|    -0.11|
|1610612755|   76ers|  2004|     0.17|
|1610612755|   76ers|  2004|    -0.11|
|1610612755|   76ers|  2005|     0.07|
|1610612755|   76ers|  2005|    -0.08|
|1610612755|   76ers|  2006|     0.07|
|1610612755|   76ers|  2006|     -0.1|
|1610612755|   76ers|  2007|     0.08|
|1610612755|   76ers|  2007|    -0.02|
|1610612755|   76ers|  2008|     0.05|
|1610612755|   76ers|  2008|    -0.05|
|1610612755|   76ers|  2009|    -0.07|
|1610612755|   76ers|  2009|     0.04|
|1610612755|   76ers|  2010|     0.11|
|1610612755|   76ers|  2010|    -0.15|
|1610612755|   76ers|  2011|     0.15|
|1610612755|   76ers|  2011|    -0.07|
|1610612755|   76ers|  2012|     0.06|
|1610612755|   76ers|  2012|    -0.13|
+----------+--------+------+---------+
only showing top 20 rows



# SIMPSONS